In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [2]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Falaq/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Falaq/metadata_113.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,3_1.wav,1,113,Al-Falaq
1,3_2.wav,1,113,Al-Falaq
2,3_3.wav,1,113,Al-Falaq
3,3_4.wav,1,113,Al-Falaq
4,3_5.wav,1,113,Al-Falaq
5,2_1.wav,2,113,Al-Falaq
6,2_2.wav,2,113,Al-Falaq
7,2_3.wav,2,113,Al-Falaq
8,2_4.wav,2,113,Al-Falaq
9,2_5.wav,2,113,Al-Falaq


Feature Extraction

In [3]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [4]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [5]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.00236511 -0.00085449 -0.00180054 ... -0.00610352 -0.01637268
 -0.01931763] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.03146362 -0.02101135 -0.06843567 ... -0.01296997 -0.02432251
 -0.03134155] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.02662659 -0.04092407 -0.03167725 ...  0.05563354  0.07296753
  0.0741272 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.0352478  -0.04

In [6]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (5, 704, 40)
Shape of labels for fold 1: (5,)


In [7]:
mfcc_data_by_fold[3]["data"].shape

(5, 704, 40)

In [8]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [9]:
mfcc_data_by_fold[1]["data"].shape

(5, 704, 40, 1)

After Training Work, Deep Features 

In [10]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 63ms/step


In [11]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [12]:
intermediate_layer_outputs.shape

(3, 5, 1024)

In [13]:
intermediate_layer_outputs

array([[[-0.9957755 , -0.9999988 ,  0.99934167, ...,  0.99901414,
         -0.98420286, -0.99891275],
        [-0.99219346, -0.9999994 ,  0.9990255 , ...,  0.99977815,
         -0.98702705, -0.99978054],
        [-0.9866689 , -1.        ,  0.9999577 , ...,  0.99817777,
         -0.99333465, -0.9713208 ],
        [-0.99944305, -1.        ,  0.99999714, ...,  0.9997469 ,
         -0.9965239 , -0.99987   ],
        [-0.99986994, -1.        ,  0.99998814, ...,  0.99901205,
         -0.98538977, -0.9994304 ]],

       [[-0.910686  ,  0.9999993 ,  0.99899375, ...,  0.98916537,
         -0.99634403, -0.99999   ],
        [-0.90741825,  0.99994004,  0.9954457 , ...,  0.9837625 ,
         -0.99976367, -0.9999672 ],
        [-0.9821633 ,  0.9993145 ,  0.98105913, ...,  0.99201065,
         -0.99150485, -0.99276066],
        [-0.94358927,  0.9995496 ,  0.9958181 , ...,  0.99776393,
         -0.9993767 , -0.9994118 ],
        [-0.9726863 ,  0.9424539 ,  0.9646652 , ...,  0.8956393 ,
         -0.98

In [14]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [15]:
al_falaq_mean = mean_across_folds

In [16]:
al_falaq_mean.shape

(5, 1024)

In [17]:
al_falaq_mean

array([[-0.9617478 , -0.3298368 ,  0.9993784 , ...,  0.9933097 ,
        -0.9819441 , -0.9996297 ],
       [-0.9524698 , -0.21543361,  0.997894  , ...,  0.9499135 ,
        -0.9893129 , -0.9997812 ],
       [-0.9331658 ,  0.33179864,  0.99201256, ...,  0.9882378 ,
        -0.99303705, -0.98798627],
       [-0.8710239 , -0.33264947,  0.99783945, ...,  0.9940136 ,
        -0.9982342 , -0.99741966],
       [-0.9908243 ,  0.31387815,  0.9881268 , ...,  0.9468527 ,
        -0.9908602 , -0.99368984]], dtype=float32)

In [18]:
# Specify the file path where you want to save the .npy file
file_path = 'al_falaq_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_falaq_mean)

Testing of Single Audio

In [19]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/falaq_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00378418  0.00276184  0.00195312 ... -0.27514648 -0.26223755
 -0.23927307] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.19169617  0.17033386  0.1937561  ...  0.10070801  0.12083435
 -0.08909607] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.23703003 -0.2459259  -0.22750854 ... -0.06011963 -0.06655884
 -0.07369995] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.14936829 -0.0536499  -0.02426147

In [20]:
mfcc_extracted_features_single

array([[[[ -451.15018  ],
         [   79.081604 ],
         [   -9.607639 ],
         ...,
         [   -2.5038037],
         [   -1.2141647],
         [    4.6810455]],

        [[ -424.37024  ],
         [   87.22598  ],
         [  -17.794748 ],
         ...,
         [   -4.0939064],
         [   -4.3105   ],
         [    9.621576 ]],

        [[ -433.3533   ],
         [   85.33208  ],
         [  -14.622759 ],
         ...,
         [   -3.2416427],
         [   -5.342316 ],
         [    8.438074 ]],

        ...,

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
        

In [21]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 121ms/step


In [22]:
intermediate_output

array([[-0.9821633 ,  0.9993145 ,  0.981059  , ...,  0.9920106 ,
        -0.99150485, -0.9927607 ]], dtype=float32)

In [23]:
intermediate_output.shape

(1, 1024)

In [24]:
intermediate_layer_outputs

array([[[-0.9957755 , -0.9999988 ,  0.99934167, ...,  0.99901414,
         -0.98420286, -0.99891275],
        [-0.99219346, -0.9999994 ,  0.9990255 , ...,  0.99977815,
         -0.98702705, -0.99978054],
        [-0.9866689 , -1.        ,  0.9999577 , ...,  0.99817777,
         -0.99333465, -0.9713208 ],
        [-0.99944305, -1.        ,  0.99999714, ...,  0.9997469 ,
         -0.9965239 , -0.99987   ],
        [-0.99986994, -1.        ,  0.99998814, ...,  0.99901205,
         -0.98538977, -0.9994304 ]],

       [[-0.910686  ,  0.9999993 ,  0.99899375, ...,  0.98916537,
         -0.99634403, -0.99999   ],
        [-0.90741825,  0.99994004,  0.9954457 , ...,  0.9837625 ,
         -0.99976367, -0.9999672 ],
        [-0.9821633 ,  0.9993145 ,  0.98105913, ...,  0.99201065,
         -0.99150485, -0.99276066],
        [-0.94358927,  0.9995496 ,  0.9958181 , ...,  0.99776393,
         -0.9993767 , -0.9994118 ],
        [-0.9726863 ,  0.9424539 ,  0.9646652 , ...,  0.8956393 ,
         -0.98

Euclidian Distance with Mean

In [25]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_falaq_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_falaq_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_falaq_mean: {closest_index+1}")
print(f"Closest value in al_falaq_mean: {closest_value}")

Index of closest value in al_falaq_mean: 3
Closest value in al_falaq_mean: [-0.9331658   0.33179864  0.99201256 ...  0.9882378  -0.99303705
 -0.98798627]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")